# Single horizon

In [10]:
import pandas as pd
import ast

# load prediction for Moirai for 2023-2024
df_moirai = pd.read_csv("analysis/future_data/temporary_2023_2024/es_future_final_moirai_2023_2024.csv")

# drop the first context_lenght rows because they have missing values for Result (nan)
# we keep the last 3 rows even if they don't have predictions. They will serve as a reference for the last 3 predicted values.
df_moirai = df_moirai.drop(df_moirai.index[:383])

# fix the horizon for predictions
horizon = 2
horizon_index = horizon - 1

prediction_length = 3

last_index = df_moirai.index[-1]
last_index_for_es = last_index - prediction_length

spread = 1
to_dollar = 5
intraday_fixed_fees = 3
intraday_variable_fees = 0.15

list_of_gains = []
list_of_losses = []

for index_in_df, row in df_moirai.iterrows():
    current_price = float(row["Close"])
    if index_in_df > last_index_for_es:
        print("We estimate the gains for all values. We will not use the last 3 values because we don't have the real values for all horizons to evalue them.")
        break
    predictions_list = ast.literal_eval(row["Result"])
    future_prices = df_moirai.loc[index_in_df:].iloc[1:prediction_length+1]["Close"].tolist()
    
    predicted_price = predictions_list[horizon_index]
    future_price = future_prices[horizon_index]
    
    if predicted_price > current_price: # we predict a UP
        difference = 0
        if future_price > current_price:
            difference = future_price - current_price
            difference = difference - spread
            difference = difference * to_dollar
            difference = difference - (difference * intraday_variable_fees) - intraday_fixed_fees
            list_of_gains.append((index_in_df, difference))
        elif future_price < current_price:
            difference = future_price - current_price
            difference = difference - spread
            difference = difference * to_dollar
            difference = difference - (abs(difference) * intraday_variable_fees) - intraday_fixed_fees
            list_of_losses.append((index_in_df, difference))
        else:
            print("No gain or loss. Except for the fees.")

    elif predicted_price < current_price: # we predict a DOWN
        difference = 0
        if future_price < current_price:
            difference = current_price - future_price
            difference = difference - spread
            difference = difference * to_dollar
            difference = difference - (difference * intraday_variable_fees) - intraday_fixed_fees
            list_of_gains.append((index_in_df, difference))
        elif future_price > current_price:
            difference = current_price - future_price
            difference = difference - spread
            difference = difference * to_dollar
            difference = difference - (abs(difference) * intraday_variable_fees) - intraday_fixed_fees
            list_of_losses.append((index_in_df, difference))
        else:
            print("No gain or loss. Except for the fees.")
        
    else: # we predict a STAY
        print("We should have a stay position.")

print("Processing finished.")


total_gain = sum([gain for index, gain in list_of_gains])
total_loss = sum([loss for index, loss in list_of_losses])
result = total_gain + total_loss
total_predictions = len(list_of_gains) + len(list_of_losses)
mean_result = result / total_predictions

print("Model: ", "moirai")
print("Horizon (in hours): ", horizon)
print("Spread (in points): ", spread)
print(f"Total gain (in dollars): {total_gain}")
print(f"Total loss (in dollars): {total_loss}")

print("Number of gains: ", len(list_of_gains))
print("Number of losses: ", len(list_of_losses))

print(f"Result: {result} points")

if total_gain > abs(total_loss):
    print(f"Mean result (positive) {mean_result} dollars")
else:
    print(f"Mean result (negative) {mean_result} dollars")

# Generate Excel File
data = {
    "Model": ["moirai"],
    "Horizon (in hours)": [horizon],
    "Spread (in points)": [spread],
    "Total gain (in dollars)": [total_gain],
    "Total loss (in dollars)": [total_loss],
    "Number of gains": [len(list_of_gains)],
    "Number of losses": [len(list_of_losses)],
    "Result (in dollars)": [result],
    "Mean result (per prediction, in dollars)": [mean_result]
}

df = pd.DataFrame(data)

# Save to Excel
file_path = "model_performance_dollars.xlsx"
df.to_excel(file_path, index=False, engine='openpyxl')
print(f"Data saved to {file_path}")


No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or loss. Except for the fees.
No gain or l

# Multiple horizons

In [15]:
import pandas as pd
import ast
from math import prod

# 1. Load the CSV (same file for all horizons)
df_moirai = pd.read_csv("analysis/future_data/temporary_2023_2024/es_future_final_moirai_2023_2024.csv")

# 2. Drop the first context_length rows because they have missing values for Result (NaN)
#    Keep the last 3 rows as a reference for the last 3 predicted values
df_moirai = df_moirai.drop(df_moirai.index[:383])

# 3. Shared configuration for all horizons
prediction_length = 3
spread = 1
to_dollar = 5
intraday_fixed_fees = 3
intraday_variable_fees = 0.15
leverage = 8

# 4. We'll store each horizon’s outcome in this list
results = []

# 5. Loop over horizons 1 through 3
for horizon in [1, 2, 3]:
    print("\nProcessing horizon:", horizon)
    horizon_index = horizon - 1

    last_index = df_moirai.index[-1]
    last_index_for_es = last_index - prediction_length

    list_of_net_gains = []
    list_of_net_losses = []

    # 6. Iterate through DataFrame rows
    for index_in_df, row in df_moirai.iterrows():
        current_price = float(row["Close"])
        if index_in_df > last_index_for_es:
            print("We estimate the gains for all values. We will not use the last 3 values "
                  f"because we don't have the real values for all horizons (H={horizon}).")
            break

        # Convert string from 'Result' column into a list of future predicted prices
        predictions_list = ast.literal_eval(row["Result"])
        # Get actual future prices
        future_prices = df_moirai.loc[index_in_df:].iloc[1 : prediction_length + 1]["Close"].tolist()

        predicted_price = predictions_list[horizon_index]
        future_price = future_prices[horizon_index]

        gross_gain = 0
        difference = 0

        # Predict UP
        if predicted_price > current_price:
            if future_price > current_price: # correct prediction
                difference = future_price - current_price
            elif future_price < current_price: # wrong prediction
                difference = future_price - current_price

        # Predict DOWN
        elif predicted_price < current_price:
            if future_price < current_price: # correct prediction
                difference = current_price - future_price
            elif future_price > current_price: # wrong prediction
                difference = current_price - future_price
        # Predict STAY
        else:
            print("We should have a stay position.")

        current_price_dollars = current_price * to_dollar
        gross_difference = difference - spread
        gross_difference *= to_dollar
        net_difference = gross_difference - (abs(gross_difference) * intraday_variable_fees) - intraday_fixed_fees # abs() because it can be negative

        if net_difference > 0:
            list_of_net_gains.append((index_in_df, net_difference, current_price_dollars))
        elif net_difference < 0:
            list_of_net_losses.append((index_in_df, net_difference, current_price_dollars))

    print(f"Processing finished for horizon {horizon}.")

    # 7. Calculate totals for this horizon
    total_net_gain = sum([net_gain for index, net_gain, current_price_dollars in list_of_net_gains])
    total_net_loss = sum([net_loss for index, net_loss, current_price_dollars in list_of_net_losses])
    net_result = total_net_gain + total_net_loss
    total_predictions_number = len(list_of_net_gains) + len(list_of_net_losses)
    mean_net_result = net_result / total_predictions_number

    list_of_rates = []
    
    # 8. Calculate performance geometric mean
    for index, net_gain, current_price_dollars in list_of_net_gains:
        assert net_gain > 0, "Gain should be positive"
        ratio_price_leverage = current_price_dollars / leverage
        perf_rate = net_gain / current_price_dollars
        list_of_rates.append(perf_rate)
    for index, net_loss, current_price_dollars in list_of_net_losses:
        assert net_loss < 0, "Loss should be negative"
        ratio_price_leverage = current_price_dollars / leverage
        perf_rate = net_loss / current_price_dollars
        list_of_rates.append(perf_rate)

    geometric_mean = prod(1 + r for r in list_of_rates) ** (1 / len(list_of_rates))
    print("Geometric mean of performance rates:", geometric_mean)

    # 8. Print summary for this horizon
    print("Model:", "moirai")
    print("Horizon (in hours):", horizon)
    print("Spread (in points):", spread)
    print("Leverage: ", leverage)
    print(f"Intraday fixed fees: ${intraday_fixed_fees}")
    print(f"Intraday variable fees: {intraday_variable_fees*100}%")
    print("The position equals to 1 contract.")
    print(f"Total net gain (in dollars): {total_net_gain}")
    print(f"Total net loss (in dollars): {total_net_loss}")
    print("Number of gains:", len(list_of_net_gains))
    print("Number of losses:", len(list_of_net_losses))
    print("Geometric mean of performance rates:", geometric_mean)
    print(f"Net gain/loss: {net_result} dollars")

    if total_net_gain > abs(total_net_loss):
        print(f"Mean net gain {mean_net_result} dollars")
    else:
        print(f"Mean net loss {mean_net_result} dollars")

    # 9. Store results for this horizon
    results.append({
        "Model": "moirai",
        "Horizon (in hours)": horizon,
        "Spread (in points)": spread,
        "Leverage": leverage,
        "Intraday fixed fees (in $)": intraday_fixed_fees,
        "Intraday variable fees (in %)": intraday_variable_fees,
        "The position equals to 1 contract": True,
        "Geometric mean of performance rates": geometric_mean,
        "Total net gain (in $)": total_net_gain,
        "Total net loss (in $)": total_net_loss,
        "Number of gains": len(list_of_net_gains),
        "Number of losses": len(list_of_net_losses),
        "Result net (in $)": net_result,
        "Mean result net (per prediction, in $)": mean_net_result,
        "Total predictions": total_predictions_number,
    })

# 10. Convert all horizon results into a single DataFrame
df_results = pd.DataFrame(results)

# 11. Save to Excel (all horizons)
file_path = "model_performance_dollars.xlsx"
df_results.to_excel(file_path, index=False, engine='openpyxl')
print(f"Data saved to {file_path}")


Processing horizon: 1
We should have a stay position.
We estimate the gains for all values. We will not use the last 3 values because we don't have the real values for all horizons (H=1).
Processing finished for horizon 1.
Geometric mean of performance rates: 0.9998779964245542
Model: moirai
Horizon (in hours): 1
Spread (in points): 1
Leverage:  8
Intraday fixed fees: $3
Intraday variable fees: 15.0%
The position equals to 1 contract.
Total net gain (in dollars): 117333.8794642857
Total net loss (in dollars): -163714.5841464075
Number of gains: 4763
Number of losses: 10906
Geometric mean of performance rates: 0.9998779964245542
Net gain/loss: -46380.704682121795 dollars
Mean net loss -2.9600296561440933 dollars

Processing horizon: 2
We estimate the gains for all values. We will not use the last 3 values because we don't have the real values for all horizons (H=2).
Processing finished for horizon 2.
Geometric mean of performance rates: 0.9999273092501089
Model: moirai
Horizon (in hour